In [ ]:
!pip install -U "transformers==4.40.0" --upgrade
!pip install accelerate bitsandbytes

In [ ]:
!pip install accelerate

In [2]:
!pip install transformers -U

  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/a7/03/fb50fc03f86016b227a967c8d474f90230c885c0d18f78acdfda7a96ce56/tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
DEPRECATION: pytorch-lightning 1.6.0 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninsta

In [ ]:
!pip install tokenizers==0.13.3

In [ ]:
import transformers
import torch

model_id = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/u/sebono/miniconda3/envs/.py311env/lib/python3.11/site-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizerFast

# Load the model and tokenizer with device map
model_id = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"
model = LlamaForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = LlamaTokenizerFast.from_pretrained(model_id)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file yo

In [5]:
import torch
from tqdm import tqdm
from torch.nn.functional import pad

def p1(dialog):
    
    max_length = model.config.max_position_embeddings
    stride = 1
    
    pad_token_id = 0
    encodings = tokenizer(" ".join(dialog), return_tensors="pt")
    seq_len = encodings.input_ids.size(1)
    padding_len = max_length -1 
    padded_input_ids = pad(torch.tensor([], dtype=torch.long), (0, padding_len), value=pad_token_id).unsqueeze(dim=0)
    encodings.input_ids = torch.cat([padded_input_ids, encodings.input_ids], dim=1)
    seq_len = encodings.input_ids.size(1)
    
    nlls = []
    prev_end_loc = padding_len
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from max_length on the last loop 
        begin_loc = max(padding_len, begin_loc)
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood.item())

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    return nlls

In [4]:
import pandas as pd
import os
import re

data_path = "/u/sebono/conversational_dominance/data/processed/CANDOR/conversations.csv"
index_path = "/u/sebono/conversational_dominance/data/processed/CANDOR/group_1.csv"
output_path ="/u/sebono/conversational_dominance/notebooks/dominance_scores_topical.pkl"

print(f"Loaded model: {model_id}")
# Load dataset from the specified data_path
dataset = pd.read_csv(data_path)
indices = pd.read_csv(index_path)["conversation_id"]

filtered_d = dataset[dataset["file_name"].isin(indices)]

pattern='<(SPK[1-9]|MOD)>'
tot_n = len(filtered_d['file_content'])
for idx, (el, path) in enumerate(zip(filtered_d["file_content"], filtered_d["file_name"])):
    ppl = {}
    if os.path.exists(f"{output_path}/dominance_scores_{path}.pkl"):
        print(f"skipping {path} ...")
        continue

    dialog = re.sub(r'\<', r'\n<', el).split("\n")[1:]
    dialog_no_s = [re.sub(pattern,"", d) for d in dialog]
    perpl = p1(dialog_no_s)
    ppl[path] = perpl
    with open(f"{output_path}/dominance_p1_{path}.pkl", 'wb') as file:
        pickle.dump(ppl, file)

/u/sebono/miniconda3/envs/.py311env/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loaded model: unsloth/Meta-Llama-3.1-8B-bnb-4bit


  0%|                                                                                                                                                                        | 0/141021 [00:00<?, ?it/s]


NameError: name 'device' is not defined

In [ ]:
import torch
from tqdm import tqdm
from torch.nn.functional import pad
import numpy as np

def p2(dialog):
    
    max_length = model.config.n_positions
    stride = 1
    #encodings = tokenizer(" ".join(dialog), return_tensors="pt")
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in dialog]
    encodings = tokenizer(" ".join(dialog), return_tensors="pt")
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    
    assert tokens_ids_per_sentence[-1] == encodings.input_ids.size(1)
    
    pad_token_id = 0    
    padding_len = max_length -1 
    
    tokens_ids_per_sentence+=padding_len
    padded_input_ids = pad(torch.tensor([], dtype=torch.long), (0, padding_len), value=pad_token_id).unsqueeze(dim=0)
    encodings.input_ids = torch.cat([padded_input_ids, encodings.input_ids], dim=1)
    seq_len = encodings.input_ids.size(1)
    #seq_len_tter = encodings.input_ids.size(1)
    seq_len_tter = tokens_ids_per_sentence 
    nlls = []
    
    begin_loc_win=padding_len
    i=0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        begin_loc = max(padding_len, begin_loc)
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-begin_loc_win] = -100
        #print([tokenizer.decode(token, skip_special_tokens=True) for token in encodings.input_ids[:, begin_loc_win:end_loc]])
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood.item())
        if seq_len_tter[i] == end_loc:
            begin_loc_win = seq_len_tter[i]
            i+=1
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    
    return nlls

In [ ]:
import pandas as pd
import os
import re
import pickle

data_path = "/u/sebono/conversational_dominance/data/processed/multisimo/conversations.csv"
index_path = "/u/sebono/conversational_dominance/data/processed/multisimo/group_2.csv"
output_path ="/u/sebono/conversational_dominance/notebooks/information_exchange_labelling/dataset_perplexity_results/multisimo_p2/"

print(f"Loaded model: {model_id}")
# Load dataset from the specified data_path
dataset = pd.read_csv(data_path)
indices = pd.read_csv(index_path)["conversation_id"]

filtered_d = dataset[dataset["file_name"].isin(indices)]

pattern='<(SPK[1-9]|MOD)>'
tot_n = len(filtered_d['file_content'])
for idx, (el, path) in enumerate(zip(filtered_d["file_content"], filtered_d["file_name"])):
    ppl = {}
    dialog = re.sub(r'\<', r'\n<', el).split("\n")[1:]
    dialog_no_s = [re.sub(pattern, ">", d) for d in dialog]
    perpl = p2(dialog_no_s)
    ppl[path] = perpl
    with open(f"{output_path}/dominance_scores_{path}.pkl", 'wb') as file:
        pickle.dump(ppl, file)

In [ ]:
import torch
from tqdm import tqdm
from torch.nn.functional import pad

def p3(dialog):
    
    max_length = model.config.n_positions
    stride = 1
    
    pad_token_id = 0
    encodings = tokenizer(" ".join(dialog), return_tensors="pt")
    encoding_nxt_line = tokenizer("\n", return_tensors="pt").input_ids.to(device)
    seq_len = encodings.input_ids.size(1)
    padding_len = max_length -1 
    padded_input_ids = pad(torch.tensor([], dtype=torch.long), (0, padding_len), value=pad_token_id).unsqueeze(dim=0)
    encodings.input_ids = torch.cat([padded_input_ids, encodings.input_ids], dim=1)
    seq_len = encodings.input_ids.size(1)
    
    nlls = []
    prev_end_loc = padding_len
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc # may be different from max_length on the last loop 
        begin_loc = max(padding_len, begin_loc)
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        input_ids_xt = torch.cat([input_ids, encoding_nxt_line], dim=1)
        target_ids = input_ids_xt.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids_xt, labels=target_ids)
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood.item())

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break
    return nlls

In [ ]:
import pandas as pd
import os
import re
import pickle

data_path = "/u/sebono/conversational_dominance/data/processed/topical/conversations.csv"
index_path = "/u/sebono/conversational_dominance/data/processed/topical/group_1.csv"
output_path ="/u/sebono/conversational_dominance/notebooks/information_exchange_labelling/dataset_perplexity_results/topical_per_utterance/"

print(f"Loaded model: {model_id}")
# Load dataset from the specified data_path
dataset = pd.read_csv(data_path)
indices = pd.read_csv(index_path)["conversation_id"]

filtered_d = dataset[dataset["file_name"].isin(indices)]

pattern='<(SPK[1-9]|MOD)>'
tot_n = len(filtered_d['file_content'])
for idx, (el, path) in enumerate(zip(filtered_d["file_content"], filtered_d["file_name"])):
    ppl = {}
    if os.path.exists(f"{output_path}/dominance_scores_{path}.pkl"):
        print(f"skipping {path} ...")
        continue
    dialog = re.sub(r'\<', r'\n<', el).split("\n")[1:]
    dialog_no_s = [re.sub(pattern, "", d) for d in dialog]
    perpl = p3(dialog_no_s)
    ppl[path] = perpl
    with open(f"{output_path}/dominance_scores_{path}.pkl", 'wb') as file:
        pickle.dump(ppl, file)

In [ ]:
import pandas as pd
import os
import re

data_path = "/u/sebono/conversational_dominance/data/processed/topical/conversations.csv"
index_path = "/u/sebono/conversational_dominance/data/processed/topical/group_1.csv"
output_path ="/u/sebono/conversational_dominance/notebooks/dominance_scores_topical.pkl"

print(f"Loaded model: {model_id}")
# Load dataset from the specified data_path
dataset = pd.read_csv(data_path)
indices = pd.read_csv(index_path)["conversation_id"]

filtered_d = dataset[dataset["file_name"].isin(indices)]


tot_n = len(filtered_d['file_content'])
for idx, (el, path) in enumerate(zip(filtered_d["file_content"], filtered_d["file_name"])):
    ppl = {}
    if os.path.exists(f"{output_path}/dominance_scores_{path}.pkl"):
        print(f"skipping {path} ...")
        continue
    dialog = re.sub(r'\<', r'\n<', el).split("\n")[1:]
    dialog_no_s = [re.sub(pattern, "", d) for d in dialog]
    perpl = p2(dialog_no_s)
    ppl[path] = perpl
    with open(f"{output_path}/dominance_p1_{path}.pkl", 'wb') as file:
        pickle.dump(ppl, file)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
STOP_WORDS = set(stopwords.words('english'))

# Assuming 'matches', 'dialog', 'offset', and 'perpl' are defined earlier in your code
def perplexity_to_info(dialog, tokens, perpl, answers, pattern = '<(SPK[1-9]|MOD)>'):

    matches = re.findall(pattern, "".join(dialog))
    unique_matches = np.unique(matches)
    #encodings = tokenizer(" ".join(dialog), return_tensors="pt")
    encodings = torch.cat(tokens)
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    assert tokens_ids_per_sentence[-1] == len(encodings)
    assert len(encodings) == len(perpl)
    
    ppl_to_info = []
    prev_idx_pp = 0
    for idx, (match,answer) in enumerate(zip(matches,answers)):
        idx_pp = tokens_ids_per_sentence[idx]
        patt = matches[idx]
        label = re.sub(r'\[([^\]]+)\]: ', '', dialog[idx])
        tokens = encodings[prev_idx_pp:idx_pp]
        decoded = [tokenizer.decode([token], skip_special_tokens=True) for token in tokens]
        perpl_per_sent = perpl[prev_idx_pp:idx_pp]
        mean_value=np.nanmean(np.asarray(perpl_per_sent))
        prev_idx_pp=idx_pp
        ppl_to_info.append({"label":answer, "perpl": np.asarray(perpl_per_sent)})
    
    return ppl_to_info

def filter_out_common_words(words, perpl):
    # Remove stop words and corresponding perplexity values
    filtered_words = [word for word in words if word.lower().strip() not in STOP_WORDS]
    filtered_word_indices = [i for i, word in enumerate(words) if word.lower().strip() not in STOP_WORDS]
    assert len(list(np.asarray(perpl)[filtered_word_indices])) == len(filtered_words)
    return list(np.asarray(perpl)[filtered_word_indices]), filtered_word_indices

def perplexity_to_info_filtering_out_common_words(dialog, tokens, perpl, answers, matches, pattern = '<(SPK[1-9]|MOD)>'):
    
    encodings = torch.cat(tokens)
    unique_matches = np.unique(matches)
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    assert tokens_ids_per_sentence[-1] == len(encodings)
    assert len(encodings) == len(perpl)
    
    
    ppl_to_info = []
    prev_idx_pp = 0
    dialog_filtered = []
    for idx, (match,answer) in enumerate(zip(matches,answers)):
        idx_pp = tokens_ids_per_sentence[idx]
        patt = matches[idx]
        label = re.sub(pattern, '', dialog[idx])
        tokens = encodings[prev_idx_pp:idx_pp]
        decoded = [tokenizer.decode([token], skip_special_tokens=True) for token in tokens]
        assert len(decoded) == len(perpl[prev_idx_pp:idx_pp])
        perpl_per_sent, filtered_word_indices = filter_out_common_words(decoded, perpl[prev_idx_pp:idx_pp])
        assert len(perpl_per_sent) == len(tokens[filtered_word_indices])
        dialog_filtered.append(tokens[filtered_word_indices])
        mean_value=np.nanmean(np.asarray(perpl_per_sent))
        prev_idx_pp=idx_pp
        ppl_to_info.append({"label":answer, "perpl": np.asarray(perpl_per_sent)})
        
    return dialog_filtered, ppl_to_info

def compute_per_user_mean_perplexity_filtering_out_common_words(dialog, tokens, perpl, matches, pattern = '<(SPK[1-9]|MOD)>'):

    encodings = torch.cat(tokens)
    unique_matches = np.unique(matches)
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    assert tokens_ids_per_sentence[-1] == len(encodings)
    assert len(encodings) == len(perpl)
    
    prev_idx_pp = 0
    user_to_ppl = {}
    decoded_utterances = []
    for idx, match in enumerate(matches):
        idx_pp = tokens_ids_per_sentence[idx]
        patt = matches[idx]
        label = re.sub(pattern, '', dialog[idx])
        tokens = encodings[prev_idx_pp:idx_pp]
        decoded = [tokenizer.decode([token], skip_special_tokens=True) for token in tokens]
        assert len(decoded) == len(perpl[prev_idx_pp:idx_pp])
        perpl_per_sent, filtered_word_indices = filter_out_common_words(decoded, perpl[prev_idx_pp:idx_pp])
        decoded_utterances.append([tokenizer.decode([token], skip_special_tokens=True) for token in tokens[filtered_word_indices]])
        assert len(perpl_per_sent) == len(tokens[filtered_word_indices])
        mean_value=np.nanmean(np.asarray(perpl_per_sent))
        prev_idx_pp=idx_pp
        if patt not in user_to_ppl:
            user_to_ppl[patt] = []
        user_to_ppl[patt].append(mean_value)
        
    return user_to_ppl, decoded_utterances

def compute_per_user_mean_perplexity(dialog, tokens, perpl, matches, pattern = '<(SPK[1-9]|MOD)>'):
    
    encodings = torch.cat(tokens)
    #matches = re.findall(pattern, "".join(dialog))
    unique_matches = np.unique(matches)
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    assert tokens_ids_per_sentence[-1] == len(encodings)
    assert len(encodings) == len(perpl)


    prev_idx_pp = 0
    user_to_ppl = {}
    for idx in range(len(matches)):
        idx_pp = tokens_ids_per_sentence[idx]
        patt = matches[idx]
        tokens = encodings[prev_idx_pp:idx_pp]
        decoded = [tokenizer.decode([token], skip_special_tokens=True) for token in tokens]
        perpl_per_sent = perpl[prev_idx_pp:idx_pp]
        mean_value=np.nanmean(np.asarray(perpl_per_sent))
        if patt not in user_to_ppl:
            user_to_ppl[patt] = []
        user_to_ppl[patt].append(mean_value)
        
    return  user_to_ppl
    
def compute_graph_perplexity(tokens, p1,p2, matches, pattern = '<(SPK[1-9]|MOD)>', answers=None):
    
    dialog = [tokenizer.decode(token, skip_special_tokens=True) for token in tokens]
    unique_matches = np.unique(matches)
    
    rows = int(np.ceil(np.sqrt(len(dialog))))
    # Create an 8x8 grid of subplots
    fig, axes = plt.subplots(rows, rows, figsize=(30, 30))
    num_plots = len(dialog)
    # Set smaller font size
    plt.rcParams.update({'font.size': 8})
    
    assert num_plots == len(matches)
    encodings = torch.cat(tokens)
    tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
    assert tokens_ids_per_sentence[-1] == len(p1)
    
    prev_idx_pp = 0
    for idx, ax in enumerate(axes.flatten()):
        if idx < num_plots:
            idx_pp = tokens_ids_per_sentence[idx]
            patt = matches[idx]
            tokens = encodings[prev_idx_pp:idx_pp]
            decoded = [tokenizer.decode([token], skip_special_tokens=True) for token in tokens]
            p1_per_sent = p1[prev_idx_pp:idx_pp]
            p2_per_sent = p2[prev_idx_pp:idx_pp]
            ax.plot(np.asarray(p1_per_sent), label=f'{patt} p1')
            ax.plot(np.asarray(p2_per_sent), label=f'{patt} p2', color='r')
            #mean_value=np.nanmean(np.asarray(perpl_per_sent))
            ax.axhline(p1_per_sent[0], color='g', label=f'{patt} p3')  # Fixed the color argument
            ax.set_xticks(np.arange(len(decoded)))
            ax.set_xticklabels(decoded, rotation=90)
            if answers is not None:
                ax.set_title(f'{answers[idx]}')
            ax.legend()
            prev_idx_pp=idx_pp

    # Hide any remaining empty subplots
    for ax in axes.flatten()[num_plots:]:
        ax.axis('off')
        
    plt.subplots_adjust(hspace=0.5, top=0.95)  
    plt.suptitle("Per-Word Perplexity across the Dataset", fontsize=30)
    plt.legend()
    plt.show()

### Multisimo

In [ ]:
import numpy as np
import re
import pandas as pd

In [ ]:
multisimo_df = pd.read_csv("/u/sebono/conversational_dominance/data/processed/multisimo/conversations.csv")
multisimo_df

In [ ]:
# load perplexity
import os
import pickle

p1_multisimo_path = "/u/sebono/conversational_dominance/notebooks/information_exchange_labelling/dataset_perplexity_results/multisimo_p1"
perplexity_scores_list = []
perplexity_scores_list_p3 = []
for root, dirs, files in os.walk(p1_multisimo_path):
    for file in files:
        if file.endswith('.pkl'):
            file_name = file.replace("dominance_scores_","").replace(".pkl","")
            file_path = os.path.join(root, file)
            with open(file_path, 'rb') as f:
                loaded = pickle.load(f)
                perplexity_scores_list.append(loaded)

perplexity_scores_p1 = {k:v for element in perplexity_scores_list for k,v in element.items()}

In [ ]:
# load perplexity
import os
import pickle

p1_multisimo_path = "/u/sebono/conversational_dominance/notebooks/information_exchange_labelling/dataset_perplexity_results/multisimo_p2"
perplexity_scores_list = []

for root, dirs, files in os.walk(p1_multisimo_path):
    for file in files:
        if file.endswith('.pkl'):
            file_name = file.replace("dominance_scores_","").replace(".pkl","")
            file_path = os.path.join(root, file)
            with open(file_path, 'rb') as f:
                perplexity_scores_list.append(pickle.load(f))

perplexity_scores_p2 = {k:v for element in perplexity_scores_list for k,v in element.items()}

In [ ]:
perplexity_scores_p2.keys()

In [ ]:
def create_extended_list(lst, arr):
    result = []
    prev_count = 0
    for i, count in enumerate(arr):
        result.extend([lst[i]] * (count - prev_count))
        prev_count = count
    return result

def compute_p3(multisimo_df, perplexity_scores_p1):
    perplexity_scores_p3 = {}
    for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
        pattern = '<(SPK[1-9]|MOD)>'
        matches = re.findall(pattern, "".join(content))
        content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
        content = [re.sub(pattern,">", d) for d in content]
        assert len(perplexity_scores_p1[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
        tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
        assert len(torch.cat(tokens)) == len(perplexity_scores_p1[name])
        tokens_ids_per_sentence = np.cumsum([t.size(0) for t in tokens])
        value = np.asarray([0] + list(np.asarray(perplexity_scores_p1[name])[tokens_ids_per_sentence[:-1]]))
        tokens_value = np.asarray([0] + list(np.asarray(torch.cat(tokens))[tokens_ids_per_sentence[:-1]]))
        perplexity_scores_p3[name] = create_extended_list(value, tokens_ids_per_sentence)
        assert len(perplexity_scores_p3[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)

    return perplexity_scores_p3

In [ ]:
perplexity_scores_p3 = compute_p3(multisimo_df, perplexity_scores_p1)

In [ ]:
perplexity_scores_p3.keys()

In [ ]:
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores_p1[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=False).input_ids.size(1)
    encodings = tokenizer([" ".join(content)], return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    dialog_filtered_tokens, info = perplexity_to_info_filtering_out_common_words(content, tokens, perplexity_scores_p1[name],[None for _ in range(len(content))],matches, pattern=r'<([^>]+)>')
    perpl = []
    for el in info:
        perpl += list(el['perpl'])
    assert len(torch.cat(dialog_filtered_tokens)) == len(perpl)
    compute_graph_perplexity(dialog_filtered_tokens,perpl,matches,pattern=pattern) 

In [ ]:
dominance = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    dominance[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores_p1[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    assert len(torch.cat(tokens)) == len(perplexity_scores_p1[name])
    compute_graph_perplexity(tokens,perplexity_scores_p1[name],perplexity_scores_p2[name],matches, pattern=pattern) 

In [ ]:
only_non_common_words_dominance = {}
decoded_utterances = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    print(name)
    only_non_common_words_dominance[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
    encodings = tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    result, decoded = compute_per_user_mean_perplexity_filtering_out_common_words(content,tokens,perplexity_scores[name],matches,pattern=pattern)
    decoded_utterances[name] = "".join(["".join(d) for d in decoded])
    for patt in result.keys():
        if patt in np.unique(matches):
            print(f"{patt}: {np.mean(result[patt])}")
            only_non_common_words_dominance[name][patt] = np.mean(result[patt])

In [ ]:
pd.DataFrame({"file_name":decoded_utterances.keys(),"file_content":decoded_utterances.values()}).to_csv("/u/sebono/conversational_dominance/data/processed/multisimo/conversations_filter_out_common_words.csv", index=False)

In [ ]:
dominance_p2 = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    dominance_p2[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores_p2[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
    encodings = tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    result = compute_per_user_mean_perplexity(content,tokens,perplexity_scores_p2[name], matches, pattern=pattern)
    for patt in result.keys():
        if patt in np.unique(matches):
            print(f"{patt}: {np.mean(result[patt])}")
            dominance_p2[name][patt] = np.mean(result[patt])

In [ ]:
dominance_p1 = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    print(name)
    dominance_p1[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores_p1[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
    encodings = tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    result = compute_per_user_mean_perplexity(content,tokens,perplexity_scores_p1[name], matches, pattern=pattern)
    for patt in result.keys():
        if patt in np.unique(matches):
            print(f"{patt}: {np.mean(result[patt])}")
            dominance_p1[name][patt] = np.mean(result[patt])

In [ ]:
dominance_p3 = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    print(name)
    dominance_p3[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    content = [re.sub(pattern,">", d) for d in content]
    assert len(perplexity_scores_p3[name]) == tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.size(1)
    encodings = tokenizer(" ".join(content), return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    tokens = [tokenizer(token, return_tensors="pt", return_offsets_mapping=True).input_ids[0] for token in content]
    result = compute_per_user_mean_perplexity(content,tokens,perplexity_scores_p3[name], matches, pattern=pattern)
    for patt in result.keys():
        if patt in np.unique(matches):
            print(f"{patt}: {np.mean(result[patt])}")
            dominance_p3[name][patt] = np.mean(result[patt])

In [ ]:
path = "/u/sebono/conversational_dominance/data/processed/multisimo/"
with open(f'{path}/dominance_p1.pkl', 'wb') as f:
    pickle.dump(dominance_p1, f)
with open(f'{path}/dominance_p2.pkl', 'wb') as f:
    pickle.dump(dominance_p2, f)
with open(f'{path}/dominance_p3.pkl', 'wb') as f:
    pickle.dump(dominance_p3, f)

In [ ]:
path = "/u/sebono/conversational_dominance/data/processed/multisimo/"
with open(f'{path}/dominance_p1.pkl', 'rb') as f:
    dominance_p1 = pickle.load(f)
with open(f'{path}/dominance_p2.pkl', 'rb') as f:
    dominance_p2 = pickle.load(f)
with open(f'{path}/dominance_p3.pkl', 'rb') as f:
    dominance_p3 = pickle.load(f)

In [ ]:
pd.DataFrame(dominance_p1), pd.DataFrame(dominance_p2), pd.DataFrame(dominance_p3)

In [ ]:
multisimo_df_dominance = pd.read_csv("../data/processed/multisimo/transcript_dominance.csv")

In [ ]:
df_ppxl_p1 = pd.DataFrame(dominance_p1).T
df_ppxl_p1.index.name = "file_name"
final_results_ppxl_p1_multisimo = pd.merge(multisimo_df_dominance,df_ppxl_p1, on="file_name")

results = pd.DataFrame({"one":final_results_ppxl_p1_multisimo["speaker_1_dom_score"]>final_results_ppxl_p1_multisimo["speaker_2_dom_score"],"two": final_results_ppxl_p1_multisimo["SPK1"]>final_results_ppxl_p1_multisimo["SPK2"]})
sum(results["one"]==results["two"])/len(results["one"])

In [ ]:
df_ppxl_p2 = pd.DataFrame(dominance_p2).T
df_ppxl_p2.index.name = "file_name"
final_results_ppxl_p2_multisimo = pd.merge(multisimo_df_dominance,df_ppxl_p2, on="file_name")

results = pd.DataFrame({"one":final_results_ppxl_p2_multisimo["speaker_1_dom_score"]>final_results_ppxl_p2_multisimo["speaker_2_dom_score"],"two": final_results_ppxl_p2_multisimo["SPK1"]>final_results_ppxl_p2_multisimo["SPK2"]})
sum(results["one"]==results["two"])/len(results["one"])

In [ ]:
df_ppxl_p3 = pd.DataFrame(dominance_p3).T
df_ppxl_p3.index.name = "file_name"
final_results_ppxl_p3_multisimo = pd.merge(multisimo_df_dominance,df_ppxl_p3, on="file_name")

results = pd.DataFrame({"one":final_results_ppxl_p3_multisimo["speaker_1_dom_score"]>final_results_ppxl_p3_multisimo["speaker_2_dom_score"],"two": final_results_ppxl_p3_multisimo["SPK1"]>final_results_ppxl_p3_multisimo["SPK2"]})
sum(results["one"]==results["two"])/len(results["one"])

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import plotly
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)
import seaborn as sns

In [ ]:
cmp = 'algae'
def correlation_heatmap(y_cols, x_cols, full_data):
    '''
    Uses scipy.stats.spearmanr function
    Params:
    y_cols, x_cols: sets of column titles (strings)
    full_data: pandas dataframe that includes all columns listed in y_cols, x_cols
    Returns:
    corr: Spearman correlation coefficient matrix (y_cols = rows, x_cols = cols of matrix)
    fig_corr: annotated plotly heatmap of coefficients
    p: Spearman p-value matrix
    fig_p: annotated plotly heatmap of p-values
    '''
    cols = y_cols+x_cols
    all_correlations = scipy.stats.spearmanr(full_data[cols], nan_policy='omit')
    corr = all_correlations.statistic[:len(y_cols), -len(x_cols):]
    corr = pd.DataFrame(corr)
    corr.columns = x_cols
    corr.index = y_cols

    p = all_correlations.pvalue[:len(y_cols), -len(x_cols):]
    p = pd.DataFrame(p)
    p.columns = x_cols
    p.index = y_cols
    
    fig_corr = px.imshow(corr, text_auto=True, aspect='auto', color_continuous_scale='agsunset')
    fig_r2 = px.imshow(corr**2, text_auto=True, aspect='auto', color_continuous_scale='agsunset')
    fig_p = px.imshow(p, text_auto=True, aspect='auto', color_continuous_scale='gray_r')

    return corr, fig_corr, p, fig_p, fig_r2

In [ ]:
def correlation(df_final):
    corr, fig_corr, p, fig_p, fig_r2 = correlation_heatmap(col_1, col_2, df_final)
    fig_corr.show()
    fig_p.show()
    fig_r2.show()
    return corr, fig_corr, p, fig_p, fig_r2

In [ ]:
final_df = pd.concat((pd.DataFrame(dominance_p1).add_suffix("_P1",axis=0),pd.DataFrame(dominance_p2).add_suffix("_P2",axis=0),pd.DataFrame(dominance_p3).add_suffix("_P3",axis=0))).T
final_df

In [ ]:
# p1 - p2
col_1 = ["MOD_P1","SPK1_P1","SPK2_P1"] 
col_2 = ["MOD_P2","SPK1_P2","SPK2_P2"]
corr, fig_corr, p, fig_p, fig_r2 = correlation(final_df)

In [ ]:
# p1 - p3
col_1 = ["MOD_P1","SPK1_P1","SPK2_P1"] 
col_2 = ["MOD_P3","SPK1_P3","SPK2_P3"]
corr, fig_corr, p, fig_p, fig_r2 = correlation(final_df)

In [ ]:
# p2 - p3
col_1 = ["MOD_P1","SPK1_P1","SPK2_P1"] 
col_2 = ["MOD_P3","SPK1_P3","SPK2_P3"]
corr, fig_corr, p, fig_p, fig_r2 = correlation(final_df)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import plotly
import plotly.io as pio
pio.renderers.default = 'iframe'
import plotly.express as px
plotly.offline.init_notebook_mode(connected=True)
import seaborn as sns

In [ ]:
y_s = final_results_ppxl_multisimo[["speaker_1_dom_score","speaker_2_dom_score"]]
X = final_results_ppxl_multisimo[["SPK1","SPK2"]]
col_1 = list(X.keys())
col_2 = list(y_s.keys())
corr, fig_corr, p, fig_p, fig_r2 = correlation(final_results_ppxl_multisimo[["speaker_1_dom_score","speaker_2_dom_score","SPK1","SPK2"]])

In [ ]:
dialog_filtered_tokens, info = perplexity_to_info_filtering_out_common_words(d,ppl_d,answers)
perpl = []
answers = []
for el in info:
    answers.append(el['label'])
    perpl += list(el['perpl'])
compute_graph_perplexity(dialog_filtered_tokens,perpl,answers=answers)

In [ ]:
###TO BE FIXED: there are some spaces that mess up the tokenization and detokenization

for el, dataset in zip(info, multisimo_df['file_content']):
    print(f"plotting {el}")
    dataset = re.sub(r'\[', r'\n[', content).split("\n")[1:]
    perplexity = []
    for sentence in info[el]:
        perplexity += list(sentence["perpl"]) 
    compute_graph_perplexity(dataset,perplexity)

In [ ]:
for i, (info_u, dataset) in enumerate(zip(info.keys(), considered_datasets_filtered)):
    for u, d in zip(info[info_u][1:], dataset.split("\n")[1:]):
        """filtering out assertive sentences like "'[Teacher]: "Very good. Continue."',
             '[Teacher]: "Please, go ahead."',
             '[Teacher]: "That\'s correct."',
             '[Teacher]: "Very good. Continue."',"""
        per_label_ppl[u['label']] += list(u['perpl'])
        per_label_dataset[u['label']].append(d)

### Tobedone when we have labels for MULTISIMO

In [ ]:
considered_datasets =multisimo_df["file_content"]

per_label_dataset={"Information seeking":[], "Information giving":[],"same":[]}
per_label_ppl={"Information seeking":[], "Information giving":[],"same":[]}

for dataset in considered_datasets:
    dataset = dataset.split("\n")
    labels = ["same" for _ in range(len(dataset))]
    encodings = tokenizer(dataset, return_tensors="pt")
    tokens_ids_per_sentence = np.cumsum([tokenizer(p, return_tensors="pt").input_ids.size(1) for p in dialog])
    prev_idx_pp = 0
    for idx, (d, l) in enumerate(zip(dataset,labels)):
        idx_pp = tokens_ids_per_sentence[idx]
        offset_ppl = len(tokenizer(d, return_tensors="pt"))
        per_label_dataset[l].append(d)
        per_label_ppl[l] += perplexity_scores[idx][prev_idx_pp:idx_pp]
        prev_idx_pp=idx_pp

### Other Dominance Indicators

In [ ]:
def count_user_utterances(content, pattern='<(SPK[1-9]|MOD)>'):
    content = " ".join(content)
    count_utterance_p_user = {}
    count_turns_p_user = {}
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    for el in np.unique(np.array(matches)):
        all_utt_p_user = [c for c in content if el in c]
        all_utt_p_user_n = sum([len(u.split(".")) for u in all_utt_p_user])
        count_utterance_p_user[el]=len(all_utt_p_user)
        count_turns_p_user[el] = np.sum(np.asarray(matches) == el)
    return count_utterance_p_user, count_turns_p_user

In [ ]:
multisimo_df = pd.read_csv("/u/sebono/conversational_dominance/data/processed/multisimo/conversations.csv")

count_utterance_p_user_l = {}
count_turns_p_user_l = {}
for content, name in zip(multisimo_df["file_content"], multisimo_df["file_name"]):
    count_utterance_p_user_l[name] = {}
    count_turns_p_user_l[name] = {}
    print(name)
    dominance[name] = {}
    pattern = '<(SPK[1-9]|MOD)>'
    matches = re.findall(pattern, "".join(content))
    content = re.sub(r'\<', r'\n<', content).split("\n")[1:]
    count_utterance_p_user, count_turns_p_user = count_user_utterances(content, pattern=pattern)
    for cu, ct in zip(count_utterance_p_user, count_turns_p_user):
        count_utterance_p_user_l[name][cu] = count_utterance_p_user[cu]
        count_turns_p_user_l[name][ct] = count_turns_p_user[ct]
        print(f"{cu}:", count_utterance_p_user[cu],count_turns_p_user[ct])

In [ ]:
count_utterance_p_user_l

In [ ]:
multisimo_df = pd.read_csv("../data/processed/multisimo/transcript_dominance.csv")

In [ ]:
df_ppxl = pd.DataFrame(count_utterance_p_user_l).T
df_ppxl.index.name = "file_name"
final_results_ppxl_multisimo = pd.merge(multisimo_df,df_ppxl, on="file_name")

In [ ]:
final_results_ppxl_multisimo

In [ ]:
results = pd.DataFrame({"one":final_results_ppxl_multisimo["speaker_1_dom_score"]>final_results_ppxl_multisimo["speaker_2_dom_score"],"two": final_results_ppxl_multisimo["SPK1"]>final_results_ppxl_multisimo["SPK2"]})
sum(results["one"]==results["two"])/len(results["one"])

### Teacher Dominated

In [ ]:
import pandas as pd
file_path = "/u/sebono/conversational_dominance/data/results/information_labels__teacher_dominated__withneutral.csv"
datasets = pd.read_csv(file_path).reset_index()

In [ ]:
ppl = {}
for index, dataset in zip(datasets["index"],datasets["transcript"]):
    dataset = dataset.split("\n")
    pattern = r'\[([^\]]+)\]'
    matches = re.findall(pattern, "".join(dataset))    
    perpl = perplexity_of_fixedlength_models(dataset)
    ppl[index] = perpl

In [ ]:
#import pickle
#file_path = "/u/sebono/conversational_dominance/data/results/information_labels__teacher_dominated__withneutral_perplexity_scores.pikle"
#with open(file_path, 'wb') as file:
#    pickle.dump(ppl, file)

In [ ]:
import pickle
file_path = "/u/sebono/conversational_dominance/data/results/information_labels__teacher_dominated__withneutral_perplexity_scores.pikle"
with open(file_path, 'rb') as file:
    perplexity_scores = pickle.load(file)

In [ ]:
considered_indices = datasets[datasets["n_responses_match"] == True]["index"]
considered_datasets =  datasets[datasets["n_responses_match"] == True]

considered_datasets["transcript"].iloc[-1]

In [ ]:
import re

considered_indices = datasets[datasets["n_responses_match"] == True]["index"]
considered_datasets =  datasets[datasets["n_responses_match"] == True]

per_label_dataset={"Information seeking":[],"Neutral": [],  "Information giving":[]}
per_label_ppl={"Information seeking":[], "Neutral": [], "Information giving":[]}

info = {}
considered_datasets_filtered = []
for dataset, response, ppl_idx in zip(considered_datasets["transcript"],considered_datasets["response"], considered_indices):
    #encodings = [tokenizer(d, return_tensors="pt").input_ids[0] for d in dataset.split("\n")]
    dataset = dataset.split("\n")
    encodings = tokenizer(" ".join(dataset), return_tensors="pt", return_offsets_mapping=True).input_ids.ravel()
    dataset_filtered, info[ppl_idx] = perplexity_to_info_filtering_out_common_words(dataset,encodings,perplexity_scores[ppl_idx],response.split("\n"))
    considered_datasets_filtered.append(dataset_filtered)

In [ ]:
per_label_ppl

In [ ]:
per_label_dataset={"Information seeking":[],"Neutral": [],  "Information giving":[]}
per_label_ppl={"Information seeking":[], "Neutral": [], "Information giving":[]}
for i, (info_u, dataset) in enumerate(zip(info.keys(), considered_datasets_filtered)):
    for u, d in zip(info[info_u], dataset):
        """filtering out assertive sentences like "'[Teacher]: "Very good. Continue."',
             '[Teacher]: "Please, go ahead."',
             '[Teacher]: "That\'s correct."',
             '[Teacher]: "Very good. Continue."',"""
        assert len(u['perpl']) == len(d)
        per_label_ppl[u['label']] += list(u['perpl'])
        per_label_dataset[u['label']].append(d)

In [ ]:
#information giving
pattern = r'\[([^\]]+)\]'
tokens = per_label_dataset["Neutral"]
perplexity = per_label_ppl["Neutral"]
compute_graph_perplexity(tokens, perplexity,pattern=pattern)

In [ ]:
#information giving
tokens = per_label_dataset["Information giving"]
perplexity = per_label_ppl["Information giving"]
compute_graph_perplexity(tokens, perplexity)

In [ ]:
len(per_label_ppl["Information giving"]), len(per_label_dataset["Information giving"])

In [ ]:
perplexity = per_label_ppl["Neutral"]
np.nanmean(perplexity), np.nanstd(perplexity)

In [ ]:
perplexity = per_label_ppl["Information giving"]
np.nanmean(perplexity), np.nanstd(perplexity)

In [ ]:
perplexity = per_label_ppl["Information seeking"]
np.nanmean(perplexity), np.nanstd(perplexity)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming per_label_ppl is a dictionary containing data for "Information seeking" and "Information giving"

# Create a figure and axis
fig, ax = plt.subplots()

# Plot KDE using seaborn
sns.kdeplot(per_label_ppl["Information seeking"], color='blue', label='Information seeking', ax=ax, cut=0, clip=(0, 4))
sns.kdeplot(per_label_ppl["Information giving"], color='green', label='Information giving', ax=ax, cut=0, clip=(0, 4))
sns.kdeplot(per_label_ppl["Neutral"], color='yellow', label='Neutral', ax=ax, cut=0, clip=(0, 4))

# Adding labels and legend
ax.set_xlabel('Values')
ax.set_ylabel('Density')
ax.legend()

# Display the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming per_label_ppl is a dictionary containing data for "Information seeking" and "Information giving"

# Create a figure and axis
fig, ax = plt.subplots()

# Plot histograms using seaborn with density=True
sns.histplot(per_label_ppl["Information seeking"], kde=True, color='blue', label='Information seeking', ax=ax, stat='density')
sns.histplot(per_label_ppl["Information giving"], kde=True, color='green', label='Information giving', ax=ax, stat='density')
sns.histplot(per_label_ppl["Neutral"], kde=True, color='yellow', label='Neutral', ax=ax, stat='density')

# Adding labels and legend
ax.set_xlabel('Values')
ax.set_ylabel('Density')
ax.legend()

# Display the plot
plt.show()


In [ ]:
#lower perplexity
dataset = per_label_dataset["Information seeking"]
perplexity = per_label_ppl["Information seeking"]
pattern = r'\[([^\]]+)\]'
compute_graph_perplexity(dataset,perplexity)